# 🔍 Walidacja Causal Discovery vs Ground Truth

Ten notebook porównuje wyniki algorytmu discovery z prawdziwą strukturą przyczynową.

**Wymagane pliki:**
- `discovery_result.json` (z Etapu 1)
- `synthetic_data/ground_truth_metadata.json`

---

In [1]:
import json
from pathlib import Path
from IPython.display import display, HTML

print("✓ Imports ready")

✓ Imports ready


## 1. Załaduj Ground Truth

In [2]:
# Załaduj prawdziwą strukturę
with open("synthetic_data/ground_truth_metadata.json") as f:
    gt = json.load(f)

# Wyciągnij prawdziwe krawędzie
all_true_edges = [
    {
        "source": e["source"],
        "target": e["target"],
        "strength": e["true_strength"],
        "description": e["description"],
    }
    for e in gt["ground_truth"]["causal_edges"]
]

# Podziel na obserwowalne i ukryte
observable_edges = [e for e in all_true_edges if e["source"] != "criminal_intent"]
hidden_edges = [e for e in all_true_edges if e["source"] == "criminal_intent"]

print(f"📊 Ground Truth:")
print(f"   Wszystkie krawędzie: {len(all_true_edges)}")
print(f"   Obserwowalne: {len(observable_edges)}")
print(f"   Ukryte (confounder): {len(hidden_edges)}")

print(f"\n✓ Obserwowalne krawędzie (które algorytm MOŻE odkryć):")
for e in observable_edges:
    print(f"   {e['source']:30s} → {e['target']:20s} (β={e['strength']:+.2f})")

print(f"\n⚠️ Ukryte krawędzie (których algorytm NIE MOŻE zobaczyć):")
for e in hidden_edges:
    print(f"   {e['source']:30s} → {e['target']:20s} (β={e['strength']:+.2f})")

📊 Ground Truth:
   Wszystkie krawędzie: 9
   Obserwowalne: 7
   Ukryte (confounder): 2

✓ Obserwowalne krawędzie (które algorytm MOŻE odkryć):
   customer_income                → transaction_amount   (β=+0.30)
   transaction_amount             → is_fraud             (β=+0.40)
   transaction_velocity_24h       → is_fraud             (β=+0.50)
   merchant_risk_score            → is_fraud             (β=+0.35)
   is_foreign_transaction         → is_fraud             (β=+0.25)
   account_age_days               → is_fraud             (β=-0.30)
   device_fingerprint_age_days    → is_fraud             (β=-0.20)

⚠️ Ukryte krawędzie (których algorytm NIE MOŻE zobaczyć):
   criminal_intent                → transaction_velocity_24h (β=+0.70)
   criminal_intent                → is_fraud             (β=+0.60)


## 2. Załaduj wyniki Discovery

In [3]:
# Sprawdź czy plik istnieje
if not Path("discovery_result.json").exists():
    print("❌ Brak pliku discovery_result.json!")
    print("   Najpierw uruchom Etap 1 (Causal Discovery)")
else:
    with open("discovery_result.json") as f:
        discovery = json.load(f)
    
    discovered_edges = [
        {
            "source": e["source"],
            "target": e["target"],
            "strength": e.get("strength", 1.0),
        }
        for e in discovery["discovered_graph"]["edges"]
    ]
    
    print(f"🔍 Discovery Result:")
    print(f"   Odkryte krawędzie: {len(discovered_edges)}")
    print(f"   Algorytmy: {discovery.get('algorithms_used', 'N/A')}")
    
    print(f"\n   Odkryte krawędzie:")
    for e in discovered_edges:
        print(f"   {e['source']:30s} → {e['target']:20s}")

🔍 Discovery Result:
   Odkryte krawędzie: 8
   Algorytmy: N/A

   Odkryte krawędzie:
   account_age_days               → is_fraud            
   is_foreign_transaction         → is_fraud            
   device_fingerprint_age_days    → is_fraud            
   merchant_risk_score            → is_fraud            
   transaction_amount             → is_fraud            
   transaction_velocity_24h       → is_fraud            
   customer_income                → transaction_amount  
   transaction_amount             → customer_income     


## 3. Porównanie: Discovery vs Ground Truth

In [4]:
# Konwertuj na zbiory krotek dla porównania
true_set = {(e["source"], e["target"]) for e in observable_edges}
disc_set = {(e["source"], e["target"]) for e in discovered_edges}

# Oblicz metryki
TP = true_set & disc_set      # True Positives: poprawnie odkryte
FP = disc_set - true_set      # False Positives: fałszywie odkryte (spurious)
FN = true_set - disc_set      # False Negatives: pominięte

precision = len(TP) / len(disc_set) if len(disc_set) > 0 else 0
recall = len(TP) / len(true_set) if len(true_set) > 0 else 0
f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

print("=" * 60)
print("📊 WYNIKI WALIDACJI")
print("=" * 60)
print(f"\n   True Positives (poprawnie odkryte):  {len(TP)}")
print(f"   False Positives (fałszywe):          {len(FP)}")
print(f"   False Negatives (pominięte):         {len(FN)}")
print(f"\n   Precision: {precision:.1%}  (ile z odkrytych jest prawdziwych)")
print(f"   Recall:    {recall:.1%}  (ile z prawdziwych odkryto)")
print(f"   F1 Score:  {f1:.1%}  (ogólna jakość)")
print("=" * 60)

📊 WYNIKI WALIDACJI

   True Positives (poprawnie odkryte):  7
   False Positives (fałszywe):          1
   False Negatives (pominięte):         0

   Precision: 87.5%  (ile z odkrytych jest prawdziwych)
   Recall:    100.0%  (ile z prawdziwych odkryto)
   F1 Score:  93.3%  (ogólna jakość)


## 4. Szczegółowa analiza

In [5]:
# Słownik ground truth dla łatwego lookup
gt_dict = {(e["source"], e["target"]): e for e in observable_edges}

print("\n✅ POPRAWNIE ODKRYTE (True Positives):")
print("-" * 60)
if TP:
    for src, tgt in sorted(TP):
        gt_info = gt_dict.get((src, tgt), {})
        strength = gt_info.get("strength", "?")
        desc = gt_info.get("description", "")
        print(f"   {src:30s} → {tgt:20s}")
        print(f"      β={strength:+.2f} | {desc}")
else:
    print("   (brak)")

print("\n❌ FAŁSZYWIE ODKRYTE (False Positives - spurious):")
print("-" * 60)
if FP:
    # Sprawdź czy to znane spurious correlations
    spurious_info = {(s["variable_a"], s["variable_b"]): s for s in gt["ground_truth"].get("spurious_correlations", [])}
    
    for src, tgt in sorted(FP):
        print(f"   {src:30s} → {tgt:20s}")
        # Sprawdź czy to znana fałszywa korelacja
        if (src, tgt) in spurious_info:
            info = spurious_info[(src, tgt)]
            print(f"      ⚠️ ZNANA FAŁSZYWA KORELACJA: {info['reason']}")
        elif (tgt, src) in spurious_info:
            info = spurious_info[(tgt, src)]
            print(f"      ⚠️ ZNANA FAŁSZYWA KORELACJA (odwrócona): {info['reason']}")
        else:
            print(f"      ⚠️ Nieznana relacja - wymaga analizy")
else:
    print("   (brak) - świetnie!")

print("\n⚠️ POMINIĘTE (False Negatives):")
print("-" * 60)
if FN:
    for src, tgt in sorted(FN):
        gt_info = gt_dict.get((src, tgt), {})
        strength = gt_info.get("strength", "?")
        desc = gt_info.get("description", "")
        print(f"   {src:30s} → {tgt:20s}")
        print(f"      β={strength:+.2f} | {desc}")
else:
    print("   (brak) - wszystkie prawdziwe relacje odkryte!")


✅ POPRAWNIE ODKRYTE (True Positives):
------------------------------------------------------------
   account_age_days               → is_fraud            
      β=-0.30 | Starsze konto → NIŻSZE ryzyko (zaufanie)
   customer_income                → transaction_amount  
      β=+0.30 | Wyższy dochód → wyższe transakcje
   device_fingerprint_age_days    → is_fraud            
      β=-0.20 | Starszy device → NIŻSZE ryzyko
   is_foreign_transaction         → is_fraud            
      β=+0.25 | Transakcja zagraniczna → wyższe ryzyko
   merchant_risk_score            → is_fraud            
      β=+0.35 | Ryzykowny merchant → wyższe ryzyko fraudu
   transaction_amount             → is_fraud            
      β=+0.40 | Wyższa kwota → wyższe ryzyko fraudu
   transaction_velocity_24h       → is_fraud            
      β=+0.50 | Więcej transakcji → wyższe ryzyko (card testing pattern)

❌ FAŁSZYWIE ODKRYTE (False Positives - spurious):
----------------------------------------------------------

## 5. Wizualizacja porównania

In [6]:
def generate_comparison_html(true_set, disc_set, gt_dict):
    """Generate HTML comparison table."""
    
    TP = true_set & disc_set
    FP = disc_set - true_set
    FN = true_set - disc_set
    
    precision = len(TP) / len(disc_set) if len(disc_set) > 0 else 0
    recall = len(TP) / len(true_set) if len(true_set) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    # Określ kolor F1
    if f1 >= 0.9:
        f1_color = "#22c55e"  # green
        f1_label = "Excellent"
    elif f1 >= 0.75:
        f1_color = "#84cc16"  # lime
        f1_label = "Good"
    elif f1 >= 0.5:
        f1_color = "#fbbf24"  # yellow
        f1_label = "Fair"
    else:
        f1_color = "#ef4444"  # red
        f1_label = "Poor"
    
    html = f'''
    <div style="font-family:Arial,sans-serif;max-width:800px;">
        <div style="background:linear-gradient(135deg,#1e3a5f,#0f172a);padding:20px;border-radius:8px;margin-bottom:20px;">
            <h2 style="margin:0;color:#60a5fa;">🔍 Walidacja Discovery vs Ground Truth</h2>
        </div>
        
        <!-- Metryki -->
        <div style="display:flex;gap:15px;margin-bottom:20px;">
            <div style="flex:1;background:#1e293b;padding:15px;border-radius:8px;text-align:center;">
                <div style="color:#94a3b8;font-size:12px;">Precision</div>
                <div style="color:#22c55e;font-size:28px;font-weight:bold;">{precision:.0%}</div>
            </div>
            <div style="flex:1;background:#1e293b;padding:15px;border-radius:8px;text-align:center;">
                <div style="color:#94a3b8;font-size:12px;">Recall</div>
                <div style="color:#3b82f6;font-size:28px;font-weight:bold;">{recall:.0%}</div>
            </div>
            <div style="flex:1;background:#1e293b;padding:15px;border-radius:8px;text-align:center;border:2px solid {f1_color};">
                <div style="color:#94a3b8;font-size:12px;">F1 Score</div>
                <div style="color:{f1_color};font-size:28px;font-weight:bold;">{f1:.0%}</div>
                <div style="color:{f1_color};font-size:11px;">{f1_label}</div>
            </div>
        </div>
        
        <!-- Confusion matrix style -->
        <div style="display:flex;gap:15px;margin-bottom:20px;">
            <div style="flex:1;background:#166534;padding:12px;border-radius:8px;">
                <div style="color:#86efac;font-size:11px;">✓ True Positives</div>
                <div style="color:white;font-size:24px;font-weight:bold;">{len(TP)}</div>
            </div>
            <div style="flex:1;background:#991b1b;padding:12px;border-radius:8px;">
                <div style="color:#fca5a5;font-size:11px;">✗ False Positives</div>
                <div style="color:white;font-size:24px;font-weight:bold;">{len(FP)}</div>
            </div>
            <div style="flex:1;background:#92400e;padding:12px;border-radius:8px;">
                <div style="color:#fde68a;font-size:11px;">⚠ False Negatives</div>
                <div style="color:white;font-size:24px;font-weight:bold;">{len(FN)}</div>
            </div>
        </div>
    '''
    
    # Tabela szczegółowa
    html += '''
        <table style="width:100%;border-collapse:collapse;background:#1e293b;border-radius:8px;overflow:hidden;">
            <thead>
                <tr style="background:#334155;">
                    <th style="padding:10px;text-align:left;color:#94a3b8;">Krawędź</th>
                    <th style="padding:10px;text-align:center;color:#94a3b8;">Ground Truth</th>
                    <th style="padding:10px;text-align:center;color:#94a3b8;">Discovery</th>
                    <th style="padding:10px;text-align:center;color:#94a3b8;">Status</th>
                </tr>
            </thead>
            <tbody>
    '''
    
    all_edges = true_set | disc_set
    for src, tgt in sorted(all_edges):
        in_true = (src, tgt) in true_set
        in_disc = (src, tgt) in disc_set
        
        gt_mark = "✓" if in_true else "-"
        gt_color = "#22c55e" if in_true else "#64748b"
        
        disc_mark = "✓" if in_disc else "-"
        disc_color = "#22c55e" if in_disc else "#64748b"
        
        if in_true and in_disc:
            status = "✅ TP"
            status_color = "#22c55e"
        elif not in_true and in_disc:
            status = "❌ FP"
            status_color = "#ef4444"
        elif in_true and not in_disc:
            status = "⚠️ FN"
            status_color = "#fbbf24"
        else:
            status = "-"
            status_color = "#64748b"
        
        # Pobierz siłę z ground truth
        strength_info = ""
        if (src, tgt) in gt_dict:
            strength_info = f" (β={gt_dict[(src, tgt)]['strength']:+.2f})"
        
        html += f'''
            <tr style="border-bottom:1px solid #334155;">
                <td style="padding:10px;color:#e2e8f0;font-family:monospace;font-size:12px;">
                    {src} → {tgt}{strength_info}
                </td>
                <td style="padding:10px;text-align:center;color:{gt_color};font-size:16px;">{gt_mark}</td>
                <td style="padding:10px;text-align:center;color:{disc_color};font-size:16px;">{disc_mark}</td>
                <td style="padding:10px;text-align:center;color:{status_color};font-weight:bold;">{status}</td>
            </tr>
        '''
    
    html += '''
            </tbody>
        </table>
        
        <div style="margin-top:15px;padding:12px;background:#334155;border-radius:8px;">
            <p style="margin:0;color:#94a3b8;font-size:12px;">
                <b>Legenda:</b><br>
                ✅ TP = True Positive (poprawnie odkryta prawdziwa relacja)<br>
                ❌ FP = False Positive (fałszywie odkryta relacja - spurious)<br>
                ⚠️ FN = False Negative (pominięta prawdziwa relacja)
            </p>
        </div>
    </div>
    '''
    
    return html

# Wygeneruj i wyświetl
html = generate_comparison_html(true_set, disc_set, gt_dict)
display(HTML(html))

Krawędź,Ground Truth,Discovery,Status
account_age_days → is_fraud (β=-0.30),✓,✓,✅ TP
customer_income → transaction_amount (β=+0.30),✓,✓,✅ TP
device_fingerprint_age_days → is_fraud (β=-0.20),✓,✓,✅ TP
is_foreign_transaction → is_fraud (β=+0.25),✓,✓,✅ TP
merchant_risk_score → is_fraud (β=+0.35),✓,✓,✅ TP
transaction_amount → customer_income,-,✓,❌ FP
transaction_amount → is_fraud (β=+0.40),✓,✓,✅ TP
transaction_velocity_24h → is_fraud (β=+0.50),✓,✓,✅ TP


## 6. Analiza ukrytego confoundera

In [7]:
print("=" * 60)
print("🔒 ANALIZA UKRYTEGO CONFOUNDERA")
print("=" * 60)

confounders = gt["ground_truth"].get("confounders", [])

for conf in confounders:
    print(f"\n   Nazwa: {conf['name']}")
    print(f"   Opis: {conf['description']}")
    print(f"   Obserwowalny: {'Tak' if conf['observable'] else 'NIE'}")
    
    # Znajdź krawędzie od tego confoundera
    conf_edges = [e for e in all_true_edges if e["source"] == conf["name"]]
    print(f"\n   Wpływa na:")
    for e in conf_edges:
        print(f"      → {e['target']} (β={e['strength']:+.2f})")

print("\n" + "=" * 60)
print("💡 WNIOSEK:")
print("=" * 60)
print("""
   Algorytm discovery NIE MOŻE zobaczyć 'criminal_intent' bo jest
   nieobserwowalny w danych. To symuluje realny świat gdzie:
   
   1. Intencja przestępcza wpływa na liczbę transakcji (card testing)
   2. Intencja przestępcza wpływa bezpośrednio na fraud
   3. Obserwujemy korelację velocity ↔ fraud, ale część tej korelacji
      jest "zafałszowana" przez wspólną przyczynę (confounder)
   
   W interfejsie review możesz oznaczyć tę krawędź jako "confounded"
   żeby system wiedział o tym ograniczeniu.
""")

🔒 ANALIZA UKRYTEGO CONFOUNDERA

   Nazwa: criminal_intent
   Opis: Ukryty czynnik: intencja przestępcza (0-1)
   Obserwowalny: NIE

   Wpływa na:
      → transaction_velocity_24h (β=+0.70)
      → is_fraud (β=+0.60)

💡 WNIOSEK:

   Algorytm discovery NIE MOŻE zobaczyć 'criminal_intent' bo jest
   nieobserwowalny w danych. To symuluje realny świat gdzie:

   1. Intencja przestępcza wpływa na liczbę transakcji (card testing)
   2. Intencja przestępcza wpływa bezpośrednio na fraud
   3. Obserwujemy korelację velocity ↔ fraud, ale część tej korelacji
      jest "zafałszowana" przez wspólną przyczynę (confounder)

   W interfejsie review możesz oznaczyć tę krawędź jako "confounded"
   żeby system wiedział o tym ograniczeniu.



## 7. Podsumowanie i eksport

In [8]:
# Zapisz raport walidacji
validation_report = {
    "metrics": {
        "precision": round(precision, 4),
        "recall": round(recall, 4),
        "f1_score": round(f1, 4),
        "true_positives": len(TP),
        "false_positives": len(FP),
        "false_negatives": len(FN),
    },
    "true_positives": [list(e) for e in sorted(TP)],
    "false_positives": [list(e) for e in sorted(FP)],
    "false_negatives": [list(e) for e in sorted(FN)],
    "ground_truth_observable_edges": len(observable_edges),
    "ground_truth_hidden_edges": len(hidden_edges),
    "discovered_edges": len(discovered_edges),
}

with open("validation_report.json", "w") as f:
    json.dump(validation_report, f, indent=2)

print("✓ Raport zapisany: validation_report.json")
print("\n" + "=" * 60)
print("📊 PODSUMOWANIE")
print("=" * 60)
print(f"""
   Precision: {precision:.0%} - {len(TP)} z {len(disc_set)} odkrytych krawędzi jest prawdziwych
   Recall:    {recall:.0%} - {len(TP)} z {len(true_set)} prawdziwych krawędzi odkryto
   F1 Score:  {f1:.0%}
   
   Interpretacja:
   - F1 > 90%: Excellent - algorytm bardzo dobrze odkrył strukturę
   - F1 > 75%: Good - większość relacji odkryta poprawnie
   - F1 > 50%: Fair - wymaga poprawy lub ręcznej weryfikacji
   - F1 < 50%: Poor - algorytm ma problemy z tymi danymi
""")

✓ Raport zapisany: validation_report.json

📊 PODSUMOWANIE

   Precision: 88% - 7 z 8 odkrytych krawędzi jest prawdziwych
   Recall:    100% - 7 z 7 prawdziwych krawędzi odkryto
   F1 Score:  93%

   Interpretacja:
   - F1 > 90%: Excellent - algorytm bardzo dobrze odkrył strukturę
   - F1 > 75%: Good - większość relacji odkryta poprawnie
   - F1 > 50%: Fair - wymaga poprawy lub ręcznej weryfikacji
   - F1 < 50%: Poor - algorytm ma problemy z tymi danymi



---
## ➡️ Następny krok

Po walidacji przejdź do **Etapu 2 (Review)** w `quick_start.ipynb` aby:
1. Zatwierdzić poprawne krawędzie (TP)
2. Odrzucić fałszywe krawędzie (FP)
3. Oznaczyć confounded edges